In [1]:
import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

#skyler added
from keras.layers import Flatten, Dense

In [2]:
import dateutil
import glob

In [3]:

print(len(glob.glob('dataset/train/maiac/2018/*')))
print(len(glob.glob('dataset/train/maiac/2019/*')))
print(len(glob.glob('dataset/train/maiac/2020/*')))
#missing 1-- have 4259 and need 4260
print(1336 + 1460 + 1464)
print('/n')


print(len(glob.glob('dataset/train/misr/2018/*')))
print(len(glob.glob('dataset/train/misr/2019/*')))
print(len(glob.glob('dataset/train/misr/2020/*')))
#missing 3-- have 785 and need 788
print(246 + 271 + 271)
print('/n')


print(len(glob.glob('dataset/test/maiac/2017/*')))
print(len(glob.glob('dataset/test/maiac/2018/*')))
print(len(glob.glob('dataset/test/maiac/2021/*')))
#missing 24: have 2420 and need 2444
print(1436 + 96 + 912)
print('/n')


print(len(glob.glob('dataset/test/misr/2017/*')))
print(len(glob.glob('dataset/test/misr/2018/*')))
print(len(glob.glob('dataset/test/misr/2021/*')))
#have all 229
print(45 + 17 + 167)
print('/n')



1336
1460
1464
246
271
271
1436
96
912
45
17
167


In [4]:
train_labels = pd.read_csv("train_labels.csv")
grid_metadata = pd.read_csv("grid_metadata.csv")
satellite_metadata = pd.read_csv("satellite_metadata.csv")
satellite_metadata['Date'] =  pd.to_datetime(satellite_metadata['time_end'], format='%Y-%m-%d')

In [5]:
train_labels

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.400000
1,2018-02-01T08:00:00Z,A2FBI,17.000000
2,2018-02-01T08:00:00Z,DJN0F,11.100000
3,2018-02-01T08:00:00Z,E5P9N,22.100000
4,2018-02-01T08:00:00Z,FRITQ,29.800000
...,...,...,...
34307,2020-12-31T18:30:00Z,P8JA5,368.611111
34308,2020-12-31T18:30:00Z,PW0JT,294.425000
34309,2020-12-31T18:30:00Z,VXNN3,224.857143
34310,2020-12-31T18:30:00Z,VYH7U,287.000000


In [25]:
####################
# REMOVE THIS LINE #
####################
train_labels = train_labels.sample(1000, random_state=42)

In [26]:
def get_grid_data(metadata, grid_id):
    return metadata[metadata["grid_id"] == grid_id]

In [27]:
def fetch_satellite_meta(metadata, datetime, location, datatype, split):
    if location == "Delhi":
        location = "dl"
    elif location == "Taipei":
        location = "tpe"
    else:
        location = "la"

    metadata = metadata[metadata['location'] == location]
    metadata = metadata[metadata['product'] == datatype]
    metadata = metadata[metadata['split'] == split]
    dateobject = dateutil.parser.parse(datetime)
    return metadata.loc[(metadata['Date'].dt.month == dateobject.month) & 
                        (metadata['Date'].dt.day == dateobject.day) &
                        (metadata['Date'].dt.year <= dateobject.year)]

In [28]:
# Opens the HDF file
def load_data(FILEPATH):
    ds = gdal.Open(FILEPATH)
    if ds == None:
        print(FILEPATH)
    return ds

def fetch_subset(granule_id):
    ds = load_data("dataset/" + granule_id)
    try ds.GetSubDatasets()[0]
    raster = gdal.Open(ds.GetSubDatasets()[8][0]) #grid5km:cosSZA features only
    band = raster.GetRasterBand(1)
    band_arr = band.ReadAsArray()
    return band_arr

In [29]:
def fetch_training_features(grid_id, datetime, split):
    temp = get_grid_data(grid_metadata, grid_id)
    sat_met = fetch_satellite_meta(satellite_metadata, 
                               datetime, 
                               temp.iloc[0]['location'], 
                               "maiac", 
                               split)
    counter = 0
    features = None
    for i in range(len(sat_met)):
        counter+=1
        granule_id = sat_met.iloc[i]['granule_id']
        subset = fetch_subset(granule_id)
        if subset is None: #***
            continue #***
        if features is None:
            features = subset
        else:
            features+=subset
    return features/counter

In [30]:
def generate_features(train_labels, split):
    labels = []
    features = []
    for i in range(len(train_labels)):
        feature = fetch_training_features(train_labels.iloc[i]['grid_id'], train_labels.iloc[i]['datetime'], split)
        features.append(np.array(feature).reshape(-1))
        if split == "train":
            labels.append(train_labels.iloc[i]['value'])
    return np.array(features), np.array(labels)

In [31]:
features, labels = generate_features(train_labels, "train")

In [32]:
def baseline_model():
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(128, input_dim=10, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=8, batch_size=5, verbose=0)
results = cross_val_score(estimator, features, labels)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -9229.06 (1392.68) MSE
